In [21]:
import re, os
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [22]:
data_dir = "/Users/zack/Customers/ds_work/topic modeling/samples"
os.chdir(data_dir)
csv = "trading_samples.csv"
messages_df = pd.read_csv(csv, low_memory=False)
messages_df.drop_duplicates(inplace=True, ignore_index=True)
messages_df.dropna(inplace=True)
print(len(messages_df))

15578


In [23]:
# stop_words = ["i", "im", "hey", "hello", "hi", "me", "my", "myself", "we", "our", "ours", "ourselves", 
# "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", 
# "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", 
# "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
# "have", "has", "had", "having", "do", "u", "does", "did", "doing", "a", "an", "the", "and", "but", "if", 
# "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", 
# "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", 
# "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", 
# "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", 
# "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "dont", "should", 
# "now", "need", "like", "would", "get", "trying", "new", "know", "got", "use", "one", "go", "g", 
# "cant", "please"]

# maybe words?
# stopwords += ["customer", "good", "set", "thank", "thanks", "yes"]
# stopwords += ["help", "home", "looking", "mobile", "number", "phone", "store", "tmobile", "trouble", "want"]

# remove dupes
# stopwords = set(stopwords)
# stop_words = stopwords.words('english')
# stop_words.extend(["thank", "thanks", "yes", "afternoon", "morning", "evening", "trade"])

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',"you're", \
    "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', \
    'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', \
    'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', \
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', \
    'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'thank',
 'thanks',
 'yes',
 'afternoon',
 'morning',
 'evening',
 'trade']

### functions for basic work

In [ ]:
def sent_to_words(sentences):
    '''remove single characters, lower-case, simple unicode output
    deacc: removes accents and normalizes alphabet'''
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts, stop_ws):
    return [[word for word in simple_preprocess(str(sentence)) if word not in stop_ws] for sentence in texts]

def make_bigrams(texts, b_mod):
    return [b_mod[sentence] for sentence in texts]

def make_trigrams(texts, t_mod):
    return [t_mod[bigram_mod[sentence]] for sentence in texts]

def lemmatization(texts, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

### Basic pre-processing of text

In [25]:
input_texts = messages_df['text'].tolist()

# Longer user messages typically contain more than 1 topic
# Decided to limit topic classification samples to 150 characters 
input_texts = [sample for sample in input_texts if len(sample) <= 150]
sent_words = list(sent_to_words(input_texts))
len(sent_words)

13504

In [26]:
# Build the bigram phrases models
# min_count ignores words with occurances lower than the count,
# threshold for generating a phrase is defined by -
# (cnt(a, b) - min_count) * N / (cnt(a) * cnt(b)) > threshold
# with 'a' and 'b' being words, and N being the corpus vocabulary size
bigram = gensim.models.Phrases(sent_words, min_count=1000, threshold=50) # higher threshold fewer phrases.

# excract just the phrases - smaller and faster
bigram_mod = gensim.models.phrases.Phraser(bigram)


In [28]:
# Remove Stop Words
cleaned_words = remove_stopwords(sent_words, stop_words)

# lookup bigrams for cleaned text
sent_words_bigrams = make_bigrams(cleaned_words, bigram_mod)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
processor = spacy.load("en_core_web_trf", disable=['parser', 'ner'])

# lemmatization keeping only noun, adj, vb, adv
lemmatized_data = lemmatization(sent_words_bigrams, processor, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

[[]]


In [29]:
# Create Dictionary - maps normalized words to integer IDs
id2word = corpora.Dictionary(lemmatized_data)

# Term Document Frequency - bag of words with ID and count
corpus = [id2word.doc2bow(text) for text in lemmatized_data]

[[]]


In [30]:
%store cleaned_words
%store sent_words
%store sent_words_bigrams
%store processor
%store corpus
%store id2word
%store lemmatized_data
%store bigram
%store bigram_mod

Stored 'cleaned_words' (list)
Stored 'sent_words' (list)
Stored 'sent_words_bigrams' (list)
Stored 'nlp' (English)
Stored 'corpus' (list)
Stored 'id2word' (Dictionary)
Stored 'data_lemmatized' (list)
Stored 'bigram' (Phrases)
Stored 'bigram_mod' (FrozenPhrases)


In [31]:
# Build Latent Dirichlet Allocation model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           random_state=42,
                                           update_every=0,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [32]:
%store lda_model

Stored 'lda_model' (LdaModel)


In [33]:
# review topics to make sure they've been computed properly
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(27,
  '0.588*"think" + 0.027*"trade" + 0.014*"pro" + 0.012*"max" + 0.010*"value" + '
  '0.000*"gor" + 0.000*"slip" + 0.000*"differentiate" + 0.000*"pristine" + '
  '0.000*"responser"'),
 (77,
  '0.183*"phone" + 0.047*"old" + 0.039*"get" + 0.034*"new" + 0.024*"still" + '
  '0.022*"credit" + 0.019*"customer" + 0.019*"number" + 0.017*"device" + '
  '0.016*"mobile"'),
 (91,
  '0.074*"phone" + 0.038*"senior" + 0.038*"get" + 0.037*"airpod" + '
  '0.033*"line" + 0.026*"monthly" + 0.024*"issue" + 0.024*"iphone" + '
  '0.021*"plan" + 0.018*"phone_number"'),
 (95,
  '0.055*"new" + 0.046*"pro" + 0.043*"phone" + 0.043*"mobile" + 0.032*"get" + '
  '0.028*"name" + 0.028*"line" + 0.028*"store" + 0.024*"device" + '
  '0.022*"customer"'),
 (8,
  '0.076*"phone" + 0.045*"get" + 0.043*"next" + 0.043*"want" + 0.037*"max" + '
  '0.032*"glass" + 0.028*"pro" + 0.027*"back" + 0.026*"th" + 0.024*"camera"'),
 (99,
  '0.151*"phone" + 0.067*"new" + 0.048*"get" + 0.024*"old" + 0.024*"still" + '
  '0.022*"work" + 

### loose analysis of model consistency

In [34]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score - DON'T USE C_V!!
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized_data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.180563724536997

Coherence Score:  0.3345672949783372


In [35]:
# %store -r cleaned_words
# %store -r sent_words
# %store -r sent_words_bigrams
# %store -r processor
# %store -r corpus
# %store -r id2word
# %store -r lemmatized_data
# %store -r bigram
# %store -r bigram_mod
# %store -r lda_model

# %store -r trigram
# %store -r trigram_mod

In [36]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\Zach.JaffeNotier\AppData\Local\Programs\Python\Python38\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
55    -0.179886  0.044201       1        1  7.864217
63     0.050227  0.179562       2        1  7.039860
19     0.238002 -0.059978       3        1  4.126172
40    -0.095479  0.138728       4        1  3.755849
69    -0.105765 -0.026846       5        1  2.374872
...         ...       ...     ...      ...       ...
4      0.012922 -0.018132      96        1  0.212035
99     0.051738  0.025068      97        1  0.195786
24    -0.021059  0.023186      98        1  0.194286
27    -0.178617 -0.302393      99        1  0.183774
77     0.060073  0.007099     100        1  0.172899

[100 rows x 5 columns], topic_info=        Term         Freq        Total  Category  logprob  loglift
7      phone  5016.000000  5016.000000   Default  30.0000  30.0000
13    iphone  2062.000000  2062.000000   Default  29.0000  29.0000
47      work   891.000000   891.000000   Default  28.0000  28.0000
3        pro  1909.000000  1909.000000   Default  27.0000  27.0000
1        max  1598.000000  1598.000000   Default  26.0000  26.0000
..       ...          ...          ...       ...      ...      ...
64     store     0.692249   414.728985  Topic100  -4.8014  -0.0352
47      work     0.728638   891.589881  Topic100  -4.7501  -0.7494
155    order     0.625742   277.727778  Topic100  -4.9024   0.2648
1        max     0.649780  1598.262387  Topic100  -4.8647  -1.4476
43   receive     0.571076   260.652601  Topic100  -4.9938   0.2368

[4966 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1196     20  0.398223       abd
168       1  0.044941      able
168       5  0.516825      able
168       7  0.067412      able
168       8  0.022471      able
...     ...       ...       ...
834      75  0.450245        yr
1429     95  0.526034       yrs
786      67  0.658549       zip
800      28  0.655961  zip_code
1266      1  0.626566       znd

[6889 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[56, 64, 20, 41, 70, 39, 16, 42, 6, 36, 11, 19, 17, 77, 65, 40, 58, 23, 80, 50, 47, 71, 72, 1, 8, 83, 10, 13, 73, 95, 88, 74, 81, 66, 46, 55, 99, 51, 22, 12, 91, 30, 31, 68, 93, 75, 61, 90, 29, 89, 87, 98, 43, 15, 27, 33, 67, 76, 48, 24, 69, 38, 14, 26, 21, 57, 45, 86, 44, 49, 53, 32, 7, 60, 82, 54, 37, 59, 4, 84, 97, 2, 79, 63, 3, 94, 18, 9, 62, 85, 34, 96, 35, 52, 92, 5, 100, 25, 28, 78])